In [1]:
# pip install transformers torch
# pip install sentencepiece
# pip install sacremoses

In [2]:
# Download the pre-trained model
from transformers import MarianMTModel, MarianTokenizer

# Download the model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-pl-en'
model_pl_en = MarianMTModel.from_pretrained(model_name)
tokenizer_pl_en = MarianTokenizer.from_pretrained(model_name)

# Save the model and tokenizer locally
model_pl_en.save_pretrained('./opus-mt-pl-en')
tokenizer_pl_en.save_pretrained('./opus-mt-pl-en')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[63429]], 'forced_eos_token_id': 0}


('./opus-mt-pl-en/tokenizer_config.json',
 './opus-mt-pl-en/special_tokens_map.json',
 './opus-mt-pl-en/vocab.json',
 './opus-mt-pl-en/source.spm',
 './opus-mt-pl-en/target.spm',
 './opus-mt-pl-en/added_tokens.json')

In [3]:
# Load the model and tokenizer
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer from the local directory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_pl_en = MarianMTModel.from_pretrained('./opus-mt-pl-en').to(device)
tokenizer_pl_en = MarianTokenizer.from_pretrained('./opus-mt-pl-en')

print(f"The model has {model_pl_en.num_parameters():,d} parameters.")

# Function to translate text from Polish to English
def translate_polish_to_english(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    
    # Generate translation using the model
    translated_tokens = model.generate(**inputs)
    
    # Decode the translated tokens to get the translated text
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    
    return translated_text

The model has 77,138,944 parameters.


In [4]:
# Example usage
polish_text = "Dzień dobry! Jak się masz?"
english_translation = translate_polish_to_english(polish_text, model_pl_en, tokenizer_pl_en)
print(english_translation) 

Good morning, how are you?


In [5]:
import re

polish_text = """
Litwo, Ojczyzno moja! ty jesteś jak zdrowie;
Ile cię trzeba cenić, ten tylko się dowie,
Kto cię stracił. Dziś piękność twą w całej ozdobie
Widzę i opisuję, bo tęsknię po tobie.

Panno święta, co Jasnej bronisz Częstochowy
I w Ostrej świecisz Bramie! Ty, co gród zamkowy
Nowogródzki ochraniasz z jego wiernym ludem!
Jak mnie dziecko do zdrowia powróciłaś cudem

Gdy od płaczącej matki, pod Twoją opiekę
Ofiarowany martwą podniosłem powiekę;
I zaraz mogłem pieszo, do Twych świątyń progu
Iść za wrócone życie podziękować Bogu

Tak nas powrócisz cudem na Ojczyzny łono!
Tymczasem, przenoś moją duszę utęsknioną
Do tych pagórków leśnych, do tych łąk zielonych,
Szeroko nad błękitnym Niemnem rozciągnionych;

Do tych pól malowanych zbożem rozmaitem,
Wyzłacanych pszenicą, posrebrzanych żytem;
Gdzie bursztynowy świerzop, gryka jak śnieg biała,
Gdzie panieńskim rumieńcem dzięcielina pała,
A wszystko przepasane jakby wstęgą, miedzą
Zieloną, na niej zrzadka ciche grusze siedzą.
"""
split_text = re.split('\n', polish_text)

for sentence in split_text:
    sentence = sentence.strip()
    if len(sentence) > 0:
        english_translation = translate_polish_to_english(sentence, model_pl_en, tokenizer_pl_en)
        print(f'{sentence} -> {english_translation} \n')

Litwo, Ojczyzno moja! ty jesteś jak zdrowie; -> Lithuania, my homeland, you are like health; 

Ile cię trzeba cenić, ten tylko się dowie, -> As much as you need to be valued, this one will only find out, 

Kto cię stracił. Dziś piękność twą w całej ozdobie -> Who has lost you today your beauty in all its adornment 

Widzę i opisuję, bo tęsknię po tobie. -> I see and describe because I miss you. 

Panno święta, co Jasnej bronisz Częstochowy -> Miss Christmas, what are you defending Jasna Częstochowa? 

I w Ostrej świecisz Bramie! Ty, co gród zamkowy -> And in the Sighty One you glow the Gate! 

Nowogródzki ochraniasz z jego wiernym ludem! -> You protect Novgorod with his faithful people! 

Jak mnie dziecko do zdrowia powróciłaś cudem -> How did my baby recover by a miracle? 

Gdy od płaczącej matki, pod Twoją opiekę -> ♪ When from a crying mother ♪ ♪ Under your care ♪ 

Ofiarowany martwą podniosłem powiekę; -> Sacrificed dead I lifted up the eyelid; 

I zaraz mogłem pieszo, do Twych świ